<a href="https://colab.research.google.com/github/cedro3/BlendGAN/blob/main/BlendGAN_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### セットアップ

In [ ]:
#@title セットアップ

# githubからコードを取得
! git clone https://github.com/cedro3/BlendGAN.git
%cd BlendGAN

# ninjaインストール
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

# 学習済みパラメータのダウンロード
import gdown
gdown.download('https://drive.google.com/uc?id=1D27HPNOSx9kWIhc13VevRy0pUv_xYiJb', './pretrained_models/blendgan.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1pWWSm_c75ieMExJPWJuYA1wby-hm4f1J', './pretrained_models/psp_encoder.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1qshfqj8SdmgQv_kfLpiohbI3QPQF-OE5', './pretrained_models/style_encoder.pt', quiet=False)

# ランドマークデータのダウンロード
! wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
! bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

# 画像の表示関数
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = os.listdir(folder)
    files.sort()
    for i, file in enumerate(files):
        img = Image.open(folder+'/'+file)    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(folder+'/'+file, fontsize=15)               
    plt.show()
    plt.close()


### コード本体
**スタイルにする画像（jpg）をpicフォルダーに保存**\
**動画をBlendGANディレクトリに保存**

In [ ]:
#@title スタイル顔画像の切り出し
#@markdown picフォルダーの画像から顔画像を切り出しalign_styleフォルダーに保存

import os
import shutil
from tqdm import tqdm

if os.path.isdir('align_style'):
     shutil.rmtree('align_style')
os.makedirs('align_style', exist_ok=True)

def run_alignment(image_path):
  import dlib
  from alignment import align_face
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor) 
  return aligned_image 

path = './pic'
files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = run_alignment(path+'/'+file)
  input_image.resize((1024,1024))
  input_image.save('./align_style/'+file)

display_pic('align_style')

In [ ]:
#@title スタイル顔画像指定
#@markdown　align_styleフォルダーの中から画像を指定し test_imgs/style_imgsへコピー
input = "013.jpg"#@param {type:"string"}
file = './align_style/'+input

# original_imagesフォルダーリセット
if os.path.isdir('test_imgs/style_imgs'):
    shutil.rmtree('test_imgs/style_imgs')
os.makedirs('test_imgs/style_imgs', exist_ok=True)

# original_imagesフォルダーへコピー
import shutil
shutil.copy(file, 'test_imgs/style_imgs/'+input) 

In [ ]:
#@title ビデオを静止画に変換
#@markdown ルートのビデオを静止画にバラしimagesフォルダーに保存
import os
import shutil
import cv2

# imagesフォルダーリセット
if os.path.isdir('images'):
    shutil.rmtree('images')
os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= './2.mp4',   # ビデオ指定
                   image_dir='./images/', 
                   image_file='%s.jpg'):  
 
    # Initial setting
    i = 0
    interval = 1
    length = 1000  # 最大フレーム数
    
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)  # fps取得
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:
           #frame = cv2.resize(frame, (256, 256))  # 256×256にリサイズ  
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()
    return fps, i, interval
 
fps, i, interval = video_2_images()
print('fps = ', fps)
print('flames = ', i)
print('interval = ', interval)

**自分の画像を使う場合はpicフォルダーに画像をアップロードしてから下記を実行して下さい**

In [ ]:
#@title 顔画像の切り出し
#@markdown imagesフォルダーの画像から顔画像を切り出しalign_imagesに保存
import os
import shutil
from tqdm import tqdm

if os.path.isdir('align_images'):
     shutil.rmtree('align_images')
os.makedirs('align_images', exist_ok=True)

def run_alignment(image_path):
  import dlib
  from alignment import align_face
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor) 
  return aligned_image 

path = './images'
files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = run_alignment(path+'/'+file)
  input_image.resize((1024,1024))
  input_image.save('./align_images/'+file)


In [ ]:
#@title 動画のスタイルブレンド
#@markdown align_imagesフォルダーの連番画像にalign_style画像のスタイルをミックスしresults/style_tranferフォルダーに連番で保存
# style_transferフォルダーリセット
if os.path.isdir('results/style_transfer'):
    shutil.rmtree('results/style_transfer')

! python style_transfer_folder.py --size 1024 --ckpt ./pretrained_models/blendgan.pt --psp_encoder_ckpt ./pretrained_models/psp_encoder.pt --style_img_path ./test_imgs/style_imgs/ --input_img_path align_images --outdir results/style_transfer/

In [ ]:
# 画像を動画に変換
#@markdown results/style_tranferフォルダーの連番画像から動画を作成

# リセットファイル
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

!ffmpeg -r $fps -i results/style_transfer/%6d_v_$input -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
#@title 動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")